**참고문헌: 딥 러닝을 이용한 자연어 처리 입문, 02. 텍스트 전처리(Text preprocessing)**

#01. 자연어 처리 전처리 이해하기


자연어 처리는 일반적으로 토큰화, 단어 집합 생성, 정수 인코딩, 패딩, 벡터화의 과정을 거칩니다. 이번 챕터에서는 이러한 전반적인 과정에 대해서 이해합니다.

## 1. 토큰화(Tokenization)


주어진 텍스트를 단어 또는 문자 단위로 자르는 것을 토큰화라고 합니다. 예를 들어 주어진 문장이 다음과 같다고 해봅시다. 영어의 경우 토큰화를 사용하는 도구로서 대표적으로 spaCy와 NLTK가 있습니다. 물론, 파이썬 기본 함수인 split으로 토큰화를 할 수도 있습니다.

우선 영어에 대해서 토큰화 실습을 해봅시다.



In [ ]:
en_text = "A Dog Run back corner near spare bedrooms"

1. spaCy 사용하기

In [ ]:
import spacy
spacy_en = spacy.load('en_core_web_sm')

In [ ]:
def tokenize(en_text):
    return [tok.text for tok in spacy_en.tokenizer(en_text)]

In [ ]:
print(tokenize(en_text))

2. NLTK 사용하기


In [ ]:
!pip install nltk

import nltk
nltk.download('punkt')

from nltk.tokenize import word_tokenize

print(word_tokenize(en_text))

3. 띄어쓰기로 토큰화


In [ ]:
print(en_text.split())

사실 영어의 경우에는 띄어쓰기 단위로 토큰화를 해도 단어들 간 구분이 꽤나 명확하기 때문에, 토큰화 작업이 수월합니다. 하지만 한국어의 경우에는 토큰화 작업이 훨씬 까다롭습니다. 그 이유는 한국어는 조사, 접사 등으로 인해 단순 띄어쓰기 단위로 나누면 같은 단어가 다른 단어로 인식되어서 단어 집합(vocabulary)의 크기가 불필요하게 커지기 때문입니다.

단어 집합(vocabuary)이란 중복을 제거한 텍스트의 총 단어의 집합(set)을 의미합니다.
예를 들어 단어 '사과'가 많이 들어간 어떤 문장에 띄어쓰기 토큰화를 한다면 '사과가', '사과를', '사과의', '사과와', '사과는'과 같은 식으로 같은 단어임에도 조사가 붙어서 다른 단어로 인식될 수 있습니다. 예를 통해 구체적으로 이해해봅시다.

4. 한국어 띄어쓰기 토큰화

In [ ]:
kor_text = "사과의 놀라운 효능이라는 글을 봤어. 그래서 오늘 사과를 먹으려고 했는데 사과가 썩어서 슈퍼에 가서 사과랑 오렌지 사왔어"

In [ ]:
print(kor_text.split())

위의 예제에서는 '사과'란 단어가 총 4번 등장했는데 모두 '의', '를', '가', '랑' 등이 붙어있어 이를 제거해주지 않으면 기계는 전부 다른 단어로 인식하게 됩니다.

5. 형태소 토큰화

위와 같은 상황을 방지하기 위해서 한국어는 보편적으로 '형태소 분석기'로 토큰화를 합니다. 여기서는 형태소 분석기 중에서 mecab을 사용해보겠습니다. 아래의 커맨드로 colab에서 mecab을 설치합니다.

In [ ]:
! git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git 

In [ ]:
ls

In [ ]:
 cd /content/Mecab-ko-for-Google-Colab/Mecab-ko-for-Google-Colab

In [ ]:
ls

In [ ]:
 !bash install_mecab-ko_on_colab_light_220429.sh

In [ ]:
from konlpy.tag import Mecab
# from konlpy.tag import Okt, Mecab

tokenizer = Mecab()
print(tokenizer.morphs(kor_text))

앞선 예와 다르게 '의', '를', '가', '랑' 등이 전부 분리되어 기계는 '사과'라는 단어를 하나의 단어로 처리할 수 있습니다.

지금까지는 단어 또는 형태소 단위로 토큰화를 했지만 이보다도 더 작은 단위인 문자 단위로 토큰화를 수행하는 경우도 있습니다.

6. 문자 토큰화

In [ ]:
print(list(en_text))

간단히 토큰화에 대해서 알아봤습니다. 이제부터는 좀 더 많은 데이터를 가지고 실습해보겠습니다.

## 2. 단어 집합(Vocabulary) 생성


단어 집합(vocabuary)이란 중복을 제거한 텍스트의 총 단어의 집합(set)을 의미합니다. 우선, 실습을 위해서 깃허브에서 '네이버 영화 리뷰 분류하기' 데이터를 다운로드하겠습니다. 네이버 영화 리뷰 데이터는 총 20만 개의 영화 리뷰를 긍정 1, 부정 0으로 레이블링한 데이터입니다.

In [ ]:
import urllib.request
import pandas as pd
from konlpy.tag import Mecab
from nltk import FreqDist
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt", filename="ratings.txt")
data = pd.read_table('ratings.txt') # 데이터프레임에 저장
print(data.shape)
data[:10]

In [ ]:
print('전체 샘플의 수 : {}'.format(len(data)))

In [ ]:
sample_data = data[:100] # 임의로 100개만 저장

정규 표현식을 통해서 데이터를 정제합니다.

[ㄱ-ㅎ] - ㄱ부터 ㅎ까지 자음 전체

[ㅏ-ㅣ] - ㅏ부터 ㅣ까지 모음 전체

[가-힣] - 사전순으로 올 수 있는 가 부터 제일 마지막 힣까지 글자 전체


In [ ]:
sample_data['document'] = sample_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
# 한글과 공백을 제외하고 모두 제거
sample_data[:10]

형태소 분석기는 mecab을 사용합니다.

In [ ]:
# 불용어 정의
stopwords=['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

In [ ]:
tokenizer = Mecab()

In [ ]:
tokenized=[]
for sentence in sample_data['document']:
    temp = tokenizer.morphs(sentence) # 토큰화
    temp = [word for word in temp if not word in stopwords] # 불용어 제거
    tokenized.append(temp)

In [ ]:
print(tokenized[:10])

이제 단어 집합을 만들어봅시다. NLTK에서는 빈도수 계산 도구인 FreqDist()를 지원합니다.

In [ ]:
vocab = FreqDist(np.hstack(tokenized))
print('단어 집합의 크기 : {}'.format(len(vocab)))

단어를 키(key)로, 단어에 대한 빈도수가 값(value)으로 저장되어져 있습니다. vocab에 단어를 입력하면 빈도수를 리턴합니다.

In [ ]:
vocab['어릴']

'재밌'이란 단어가 총 10번 등장하였습니다. most_common()는 상위 빈도수를 가진 주어진 수의 단어만을 리턴합니다. 이를 사용하여 등장 빈도수가 높은 단어들을 원하는 개수만큼만 얻을 수 있습니다. 등장 빈도수 상위 500개의 단어만 단어 집합으로 저장해봅시다.

In [ ]:
vocab_size = 500
# 상위 vocab_size개의 단어만 보존
vocab = vocab.most_common(vocab_size)
print('단어 집합의 크기 : {}'.format(len(vocab)))

단어 집합의 크기가 500으로 줄어든 것을 확인할 수 있습니다.

# 3. 각 단어에 고유한 정수 부여


enumerate()는 순서가 있는 자료형(list, set, tuple, dictionary, string)을 입력으로 받아 인덱스를 순차적으로 함께 리턴한다는 특징이 있습니다. 인덱스 0과 1은 다른 용도로 남겨두고 나머지 단어들은 2부터 501까지 순차적으로 인덱스를 부여해봅시다.

In [ ]:
word_to_index = {word[0] : index + 2 for index, word in enumerate(vocab)}
print(word_to_index)

word_to_index['pad'] = 1
word_to_index['unk'] = 0

print(word_to_index)

이제 기존의 훈련 데이터에서 각 단어를 고유한 정수로 부여하는 작업을 진행해보겠습니다.

In [ ]:
encoded = []
for line in tokenized: #입력 데이터에서 1줄씩 문장을 읽음
    temp = []
    for w in line: #각 줄에서 1개씩 글자를 읽음
      try:
        temp.append(word_to_index[w]) # 글자를 해당되는 정수로 변환
      except KeyError: # 단어 집합에 없는 단어일 경우 unk로 대체된다.
        temp.append(word_to_index['unk']) # unk의 인덱스로 변환

    encoded.append(temp)


In [ ]:
print(encoded[:10])

# 4. 길이가 다른 문장들을 모두 동일한 길이로 바꿔주는 패딩(padding)


이제 길이가 다른 리뷰들을 모두 동일한 길이로 바꿔주는 패딩 작업을 진행해보겠습니다. 앞서 단어 집합에 패딩을 위한 토큰인 'pad'를 추가했었습니다. 패딩 작업은 정해준 길이로 모든 샘플들의 길이를 맞춰주되, 길이가 정해준 길이보다 짧은 샘플들에는 'pad' 토큰을 추가하여 길이를 맞춰주는 작업입니다.

In [ ]:
max_len = max(len(l) for l in encoded)
print('리뷰의 최대 길이 : %d' % max_len)
print('리뷰의 최소 길이 : %d' % min(len(l) for l in encoded))
print('리뷰의 평균 길이 : %f' % (sum(map(len, encoded))/len(encoded)))
plt.hist([len(s) for s in encoded], bins=50)
plt.xlabel('length of sample')
plt.ylabel('number of sample')
plt.show()

가장 길이가 긴 리뷰의 길이는 63입니다. 모든 리뷰의 길이를 63으로 통일시켜주겠습니다.

In [ ]:
for line in encoded:
    #print(line)
    if len(line) < max_len: # 현재 샘플이 정해준 길이보다 짧으면
        line += [word_to_index['pad']] * (max_len - len(line)) # 나머지는 전부 'pad' 토큰으로 채운다.

In [ ]:
print('리뷰의 최대 길이 : %d' % max(len(l) for l in encoded))
print('리뷰의 최소 길이 : %d' % min(len(l) for l in encoded))
print('리뷰의 평균 길이 : %f' % (sum(map(len, encoded))/len(encoded)))

지면의 한계로 인해 상위 3개의 샘플들만 출력해보겠습니다.

In [ ]:
print(encoded[:3])

In [ ]:


test_encode =[[1,2,3,4,5],[2,2],[44,45,34]]

for line in test_encode:
    line += [0]*5


print(test_encode)

이제 단어들을 고유한 정수로 맵핑하였으니, 각 정수를 고유한 단어 벡터로 바꾸는 작업이 필요합니다. 단어 벡터를 얻는 방법은 크게 원-핫 인코딩과 워드 임베딩이 있는데, 주로 워드 임베딩이 사용됩니다. 원-핫 인코딩과 워드 임베딩에 대해서는 다음시간에 다뤄 보겠습니다.